In [21]:
import numpy as np
import os, time
from sklearn.preprocessing import StandardScaler 

dirname = "C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/Features/Nearest_Neighbor_Noise"
if not os.path.isdir(dirname):
    os.makedirs(dirname)

X =np.load("C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/x_upsampled.npy")
y=np.load("C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/y_upsampled.npy")

In [14]:
X.shape

(1627, 256, 256, 3)

In [15]:
# def label_conv(label_arr):
#     labels=[]
#     for i in range(len(label_arr)):
#         if (label_arr[i]==[1,0]).all():
#             labels.append(0)  
#         else:
#             labels.append(1)
#     return labels

In [22]:
# Data Standardization
sc = StandardScaler()
fit = sc.fit(X)
X = fit.transform(X)

in_dist = np.zeros(y.shape[0]) + 10000
out_dist = np.zeros(y.shape[0]) + 10000
out_label = np.zeros(y.shape[0])

for cur_label in np.arange(10):
    start = time.time()
    
    print('Working with Label:', cur_label)
    cl_ind = np.where(y == cur_label)[0]
    
    print('Counting Intra Distances')
    printcounter = 0
    for u_index in cl_ind:
        printcounter += 1
        if printcounter % 500 == 0:
            print('Reporting in_dist progress', printcounter)
        
        for n_index in cl_ind:
            if u_index == n_index:
                continue
            ndist = np.linalg.norm(X[u_index] - X[n_index])
            if ndist < in_dist[u_index]:
                in_dist[u_index] = ndist
    
    other_ind = np.where(y != cur_label)[0]
    
    print('Counting Outer Distances')
    printcounter = 0
    for u_index in cl_ind:
        printcounter += 1
        if printcounter % 500 == 0:
            print('Reporting out_dist progress:', printcounter)
            
        min_dist = 10000
        min_label = 0
        point1 = X[u_index]
        for n_index in other_ind:
            ndist = np.linalg.norm(point1 - X[n_index])
            if ndist < min_dist:
                min_dist = ndist
                min_label = y[n_index]
        
        out_dist[u_index] = min_dist
        out_label[u_index] = min_label
        
    end = time.time()
    print('Total Time taken:', end-start)
    
dist_ratio = out_dist/in_dist

filename = dirname + "\\dist_ratio"
np.save(filename, dist_ratio)

filename = dirname + "\\NNlabel"
np.save(filename, out_label)

#a = np.load(dirname + "\\0_index.npy")
#b = np.load(dirname + "\\1_ratio.npy")
#c = np.load(dirname + "\\0_NNlabel.npy")

'''------ verifying how many labels have been altered actually -----'''
    
##t_y = np.load("F:\\Minhaz\\GitHubRepo\\Fall_2020\\SAT_4_6\\Features\\NAR\\_15_0.npy")
#
#change = [(train_y[i] == t_y[i]) for i in range(t_y.shape[0])]
#diff = []
#for i in range(len(change)):
#    if False in change[i]:
#        diff.append(i)
#label_num_orig = [(train_y[i][0] == 1) for i in range (train_y.shape[0])].count(True)
#print(100*len(diff)/label_num_orig, '% label noise added.')


ValueError: Found array with dim 4. StandardScaler expected <= 2.